In [1]:
import pandas as pd
import yfinance as yf
import pymongo
import time
import datetime

from datetime import datetime, timedelta
from pymongo import MongoClient

In [2]:
tickersdf = yf.download("WMT NKLA AMZN DIS FB SPCE BA AMD MSFT AAPL TSLA SPY", start = '2019-10-01', end = '2020-10-03', progress = False)
client = MongoClient('localhost', 27017)
db = client.db
comments = db.comments

commentdf = pd.DataFrame.from_records(comments.find({'created_utc': {'$exists': True}}))
commentdf.drop(commentdf[commentdf['sentiment'] == 0].index, inplace=True)
commentdf.reset_index(drop = True, inplace = True)

In [3]:
spydf = yf.download('SPY', start = '2019-10-01', end = '2020-10-03', progress = False)
dayBins = list(spydf.reset_index()['Date'])
commentdf['created_utc'] = [pd.Timestamp(datetime.utcfromtimestamp(time)) for time in commentdf['created_utc']]

In [4]:
commentdf['days'] = pd.cut(commentdf['created_utc'], bins = dayBins, labels=spydf.reset_index()['Date'][0:254])

In [263]:
data = pd.pivot_table(commentdf, values = 'id', index = ['days'], columns = ['sentiment'], aggfunc = 'count')
data.columns.name = ''
data.reset_index(inplace= True)

In [6]:
tickersdf = tickersdf.stack().swaplevel(0,1).drop(columns = ['Close', 'High', 'Low', 'Volume','Adj Close'])

In [264]:
data['tot'] = data['bullish'] + data['bearish']
data['avg'] = (data['bullish']/data['tot']).rolling(5).mean()
data['days'] = data['days'].astype('datetime64[ns]')

uploaddf = tickersdf.loc['SPY'].merge(data, left_index = True, right_on = 'days')
uploaddf['days'] = uploaddf['days'].apply(lambda x: x.value)
uploaddf.set_index('days', inplace = True)

In [265]:
lineDict['SPY'] = uploaddf.to_dict('index')

In [217]:
stocklist = ['WMT', 'NKLA', 'AMZN', 'DIS', 'FB', 'SPCE', 'BA', 'AMD', 'MSFT', 'AAPL', 'TSLA']

In [262]:
for nextStock in stocklist:
    print(nextStock)
    newdatadf = commentdf.loc[pd.DataFrame(commentdf['stocks'].tolist()).isin([nextStock]).values]

    data = pd.pivot_table(newdatadf, values = 'id', index = ['days'], columns = ['sentiment'], aggfunc = 'count')
    data.columns.name = ''
    data.reset_index(inplace= True)

    data['tot'] = data['bullish'] + data['bearish']
    data['avg'] = (data['bullish']/data['tot']).rolling(3).mean()
    data['days'] = data['days'].astype('datetime64[ns]')

    data.fillna(0, inplace=True)

    data['tot'] = data['bullish'] + data['bearish']
    data['avg'] = (data['bullish']/data['tot']).rolling(3).mean()
    data['days'] = data['days'].astype('datetime64[ns]')

    uploaddf = tickersdf.loc[nextStock].merge(data, left_index = True, right_on = 'days')
    uploaddf['days'] = uploaddf['days'].apply(lambda x: x.value)
    uploaddf.set_index('days', inplace = True)

    lineDict[nextStock] = uploaddf.to_dict('index')

WMT
NKLA
AMZN
DIS
FB
SPCE
BA
AMD
MSFT
AAPL
TSLA


In [266]:
lineDict.keys()

dict_keys(['SPY', 'WMT', 'NKLA', 'AMZN', 'FB', 'SPCE', 'BA', 'MSFT', 'TSLA', 'AMD', 'AAPL', 'DIS'])

In [ ]:
"WMT NKLA AMZN DIS FB SPCE BA AMD MSFT AAPL TSLA SPY"

In [267]:
import json
with open('result.json', 'w') as fp:
    json.dump(lineDict, fp)